# Prithvi WxC Gravity Wave: Model Fine Tuning and Inference using TerraTorch

In [ ]:
!pip install -U git+https://github.com/romeokienzler/terratorch.git@201


In [ ]:
!pip install -U albumentations # fix until https://github.com/IBM/terratorch/issues/164 is solved

In [ ]:
!pip install -U git+https://github.com/romeokienzler/gravity-wave-finetuning.git


In [ ]:
!pip install huggingface_hub

In [1]:
import terratorch # this import is needed to initialize TT's factories
from lightning.pytorch import Trainer
import os
import torch
from huggingface_hub import hf_hub_download, snapshot_download
from terratorch.models.wxc_model_factory import WxCModelFactory
import torch.distributed as dist

/usr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/romeokienzler/.local/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355' 

if dist.is_initialized():
    dist.destroy_process_group()

dist.init_process_group(
    backend='gloo',
    init_method='env://', 
    rank=0,
    world_size=1
)

In [3]:
hf_hub_download(
    repo_id="Prithvi-WxC/Gravity_wave_Parameterization",
    filename=f"magnet-flux-uvtp122-epoch-99-loss-0.1022.pt",
    local_dir=".",
)

hf_hub_download(
    repo_id="Prithvi-WxC/Gravity_wave_Parameterization",
    filename=f"config.yaml",
    local_dir=".",
)

'config.yaml'

In [4]:
hf_hub_download(
    repo_id="Prithvi-WxC/Gravity_wave_Parameterization",
    repo_type='dataset',
    filename=f"wxc_input_u_v_t_p_output_theta_uw_vw_era5_training_data_hourly_2015_constant_mu_sigma_scaling05.nc",
    local_dir=".",
)

'wxc_input_u_v_t_p_output_theta_uw_vw_era5_training_data_hourly_2015_constant_mu_sigma_scaling05.nc'

In [5]:
from prithviwxc.gravitywave.datamodule import ERA5DataModule
from terratorch.tasks.wxc_gravity_wave_task import WxCGravityWaveTask
task = WxCGravityWaveTask(WxCModelFactory())

Loading weights from magnet-flux-uvtp122-epoch-99-loss-0.1022.pt
Loaded weights


In [6]:
trainer = Trainer(
    max_epochs=1,
)
dm = ERA5DataModule(train_data_path='.', valid_data_path='.')
type(dm)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


prithviwxc.gravitywave.datamodule.ERA5DataModule

In [ ]:
results = trainer.predict(model=task, datamodule=dm, return_predictions=True)

Predicting DataLoader 0:  19%|█████████████████████████████████▏                                                                                                                                           | 9/47 [1:06:22<4:40:15,  0.00it/s]

In [ ]:
results

In [ ]:
dm.setup(stage='predict')

In [ ]:
results = trainer.train(model=task, datamodule=dm, return_predictions=True)

In [ ]:
dist.destroy_process_group()